In [1]:
import os
import os.path as osp
import glob

import pandas as pd
import matplotlib.pyplot as plt


In [2]:
ROOT = './'

csv_data = glob.glob(f"{osp.join(ROOT, 'data', '*.csv')}")

assert len(csv_data) > 0, "No csv files are found"

print(f"Found {len(csv_data)} csv files")

Found 7 csv files


# Join the CSV files together

In [3]:
AGE_CSV = '/Users/kctey/Desktop/NUS Y4S1/BN4101/UK_Biobank/data/age.csv'

df = None
for filepath in sorted(csv_data)[::-1]:
    if df is None:
        df = pd.read_csv(filepath)
    else:
        tmp_df = pd.read_csv(filepath)
        
        # inner join age dataframe
        if filepath == AGE_CSV:
            how = 'inner'
        else:
            how = 'outer'  # outer join all cognitive tests and use imputation to fill missing values

        df = df.merge(tmp_df, on='eid', how=how)

# Check Original Distribution before Imputing

In [4]:
# FI
# plt.hist(df['20191-0.0'])
# plt.show()

from scipy.stats import shapiro
shapiro(df['20191-0.0'].dropna()) # ShapiroResult(statistic=0.9766396880149841, pvalue=0.0)

# from scipy.stats import kstest
# kstest(df['20191-0.0'].dropna(), 'norm') # KstestResult(statistic=0.9823087568599015, pvalue=0.0)


/opt/homebrew/lib/python3.10/site-packages/scipy/stats/_morestats.py:1800: UserWarning: p-value may not be accurate for N > 5000.
  warnings.warn("p-value may not be accurate for N > 5000.")


ShapiroResult(statistic=0.9768627882003784, pvalue=0.0)

# Handle missing values

In [9]:
print(f"Data extracted has {len(df)} rows")

# Total 11 cognitive tests, (does not count eid and 21022)
# only keep record if user has taken more than 5 tests (7 non-NaN columns)
df = df.dropna(thresh=7)
df

Data extracted has 502411 rows


,eid,20156-0.0,20157-0.0,20247-0.0,20248-0.0,20159-0.0,20195-0.0,mean_20131,mean_20132,mean_20133,20240-0.0,31-0.0,20191-0.0,21022-0.0
0,1000085,43.202,64.591,0.0,0.0,22.0,22.0,4.500000,1.000000,22838.000000,9.0,0,6.0,51.0
1,1000158,48.845,96.175,3.0,5.0,5.0,17.0,4.500000,5.000000,35523.000000,7.0,0,5.0,60.0
2,1000251,23.656,36.673,0.0,0.0,23.0,24.0,4.500000,4.000000,14591.000000,8.0,1,10.0,53.0
3,1000279,25.344,51.384,0.0,1.0,25.0,25.0,5.666667,3.333333,21993.000000,8.0,0,4.0,54.0
4,1000295,33.183,52.936,0.0,0.0,21.0,21.0,4.500000,6.000000,32468.000000,7.0,0,6.0,57.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
121838,6009918,NaN,NaN,NaN,NaN,NaN,NaN,4.500000,6.000000,36960.000000,4.0,1,2.0,64.0
121839,6011793,NaN,NaN,NaN,NaN,NaN,NaN,4.500000,2.000000,16046.500000,7.0,0,6.0,66.0
121840,6014311,NaN,NaN,NaN,NaN,NaN,NaN,4.500000,6.500000,64201.000000,5.0,1,4.0,55.0
121841,6014493,NaN,NaN,NaN,NaN,NaN,NaN,4.500000,2.000000,21785.000000,6.0,0,6.0,64.0


In [10]:
df.dropna()

,eid,20156-0.0,20157-0.0,20247-0.0,20248-0.0,20159-0.0,20195-0.0,mean_20131,mean_20132,mean_20133,20240-0.0,31-0.0,20191-0.0,21022-0.0
0,1000085,43.202,64.591,0.0,0.0,22.0,22.0,4.500000,1.000000,22838.0,9.0,0,6.0,51.0
1,1000158,48.845,96.175,3.0,5.0,5.0,17.0,4.500000,5.000000,35523.0,7.0,0,5.0,60.0
2,1000251,23.656,36.673,0.0,0.0,23.0,24.0,4.500000,4.000000,14591.0,8.0,1,10.0,53.0
3,1000279,25.344,51.384,0.0,1.0,25.0,25.0,5.666667,3.333333,21993.0,8.0,0,4.0,54.0
4,1000295,33.183,52.936,0.0,0.0,21.0,21.0,4.500000,6.000000,32468.0,7.0,0,6.0,57.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
104009,6024594,48.132,59.984,3.0,0.0,19.0,19.0,4.500000,2.500000,32989.5,8.0,1,4.0,64.0
104010,6024683,53.382,57.337,5.0,0.0,18.0,20.0,4.500000,2.500000,21135.0,7.0,0,7.0,42.0
104011,6024781,24.867,44.819,0.0,0.0,25.0,25.0,4.500000,1.500000,14452.0,8.0,0,7.0,42.0
104012,6024822,47.510,71.331,0.0,0.0,24.0,24.0,4.500000,3.000000,23481.0,8.0,0,5.0,52.0


# Significance of each variable against age and gender

In [ ]:
# check distribution of age and gender in original dataframe



In [5]:
# Numeric Memory


In [6]:
# Pair Matching



In [7]:
# Symbol Digit Substitution



In [8]:
# Trail Making

# Data Imputation

In [11]:
df.corr()

,eid,20156-0.0,20157-0.0,20247-0.0,20248-0.0,20159-0.0,20195-0.0,mean_20131,mean_20132,mean_20133,20240-0.0,31-0.0,20191-0.0,21022-0.0
eid,1.000000,-0.001649,-0.006384,-0.002434,-0.001409,-0.000171,0.001273,-0.001727,0.004146,0.007078,-0.002459,0.001635,0.001305,0.000717
20156-0.0,-0.001649,1.000000,0.605743,0.434216,0.195227,-0.461843,-0.470475,-0.077137,0.120394,0.415023,-0.187843,-0.052731,-0.244139,0.266998
20157-0.0,-0.006384,0.605743,1.000000,0.159479,0.486708,-0.538856,-0.551675,-0.123524,0.215575,0.420025,-0.297603,-0.021030,-0.397221,0.338263
20247-0.0,-0.002434,0.434216,0.159479,1.000000,0.247376,-0.101943,-0.077428,-0.015009,0.043550,0.078594,-0.062555,-0.033092,-0.060765,0.044339
20248-0.0,-0.001409,0.195227,0.486708,0.247376,1.000000,-0.175468,-0.150001,-0.034597,0.081809,0.102429,-0.109746,-0.039255,-0.136878,0.076841
20159-0.0,-0.000171,-0.461843,-0.538856,-0.101943,-0.175468,1.000000,0.957347,0.134692,-0.214128,-0.406952,0.224857,-0.030276,0.338767,-0.429029
20195-0.0,0.001273,-0.470475,-0.551675,-0.077428,-0.150001,0.957347,1.000000,0.137771,-0.217600,-0.423677,0.221274,-0.041761,0.342691,-0.452176
mean_20131,-0.001727,-0.077137,-0.123524,-0.015009,-0.034597,0.134692,0.137771,1.000000,0.018033,0.049566,0.063307,-0.004779,0.090799,-0.122714
mean_20132,0.004146,0.120394,0.215575,0.043550,0.081809,-0.214128,-0.217600,0.018033,1.000000,0.576756,-0.108881,0.026910,-0.158036,0.172659
mean_20133,0.007078,0.415023,0.420025,0.078594,0.102429,-0.406952,-0.423677,0.049566,0.576756,1.000000,-0.157701,-0.055914,-0.233632,0.315231
